In [1]:
from IOHexperimenter import IOH_function, IOH_logger
import datetime
from ea import EA
from bfgs import BFGS
from mlsl import MLSL
import handover

In [2]:
def switch(func, budget, alg1, alg2, tau, target):
    """ Switch function to switch between two pre-defined algorithms.

    Parameters:
    -------------
    func : object
        The function that will be optimized.

    budget : int
        Budget for function evaluations, total.

    a1 : string
        The algorithm that should run first. Options: 'MLSL', 'BFGS', 'CMAES', 'EA', 'PSO'.

    a2 : string
        The algorithm that should run second. Options: 'MLSL', 'BFGS', 'CMAES', 'EA', 'PSO'.

    tau : float
        The function value at which algorithms should switch from A1 to A2.

    target : float
        The fixed-target value until which A2 should optimize

    """
    # Initialize A1
    if alg1 == 'MLSL':
        a1 = MLSL((budget/2), 'a1')
    elif alg1 == 'BFGS':
        a1 = BFGS((budget/2), 'a1')
    elif alg1 == 'EA':
        a1 = EA(((budget/2), 'a1'))
    
    if alg2 == 'MLSL':
        a2 = MLSL((budget/2), 'a2')
    elif alg2 == 'BFGS':
        a2 = BFGS((budget/2), 'a2')
    elif alg2 == 'EA':
        a2 = EA(((budget/2), 'a2'))

    # Run A1
    a1(func, tau, target)
    print(f' switch_precision so far: {func.best_so_far_precision}')

    # Handover of information
    handover.mlsl_bfgs(a1, a2)
    
    # Run A2
    a2(func, tau, target)
    print(f' switch_precision so far: {func.best_so_far_precision}')

In [5]:
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
experiment_name =  path + "simple_switch/experiment" + experiment_ID

instances = [1, 2, 3, 4, 5]

for instance in instances:
    f12 = IOH_function(12, 20, instance, suite = 'BBOB')
    l = IOH_logger(experiment_name, "run", "simple_switch_tau_30", "switch-mlsl-bfgs")
    f12.add_logger(l)
    for run in range(0, 5):
        print(f' run {run} instance {instance}')
        if run > 0:
            f12.reset()
        switch(f12, 10000, 'MLSL', 'BFGS', 30, 1e-8)
    f12.clear_logger()

 run 0 instance 1
 Init-MLSL prec: 112371637.06541933 evals: 1000
 rk update, new rk: 1.1720886322635131
 For-MLSL prec: 2.0145106416363756 evals: 2712
 MLSL prec: 2.0145106416363756 evals: 2712
 MLSL complete, x_opt: [-1.37015547  3.82495565 -0.14188379 -4.42690798 -0.1352577  -2.21255717
  0.84681665 -0.84596318  1.21861744 -0.23664743  0.92738912  2.63821876
  3.89355877  1.18672999  2.93093097 -1.89657955  1.50203223  1.66838622
 -0.33111964 -0.04444816] eval: 2712
 switch_precision so far: 2.0145106416363756
 BFGS started, x0 = [-1.37015547  3.82495565 -0.14188379 -4.42690798 -0.1352577  -2.21255717
  0.84681665 -0.84596318  1.21861744 -0.23664743  0.92738912  2.63821876
  3.89355877  1.18672999  2.93093097 -1.89657955  1.50203223  1.66838622
 -0.33111964 -0.04444816]
 BFGS prec: 2.0141118146294956 evals: 2775
 BFGS prec: 2.0140802902221995 evals: 2817
 BFGS prec: 2.014070336268697 evals: 2859
 BFGS prec: 2.0139751928312952 evals: 2901
 BFGS prec: 1.9985156652863858 evals: 2985
 B

In [ ]:
mlsl.rk

In [ ]:
mlsl()

In [ ]:
x0 = None

In [ ]:
x0

In [ ]:
if x0 != 1:
    print('yes')

In [ ]:
print(x0 is None)

In [ ]:
if x0 is None:
    print('yes')